### 371. Sum of Two Integers

- 時間複雜度：$O(1)$  
  處理的是 32 位元整數。每次迴圈，進位 b 都會至少向左移動一位。在最糟的情況下（例如 $7 + 1$ 或 $-1 + 1$），進位最多移動 32 次 就會超出範圍並消失。因為 32 是一個固定的常數，所以時間複雜度為 $O(32)$，即 $O(1)$。2. 
- 空間複雜度：$O(1)$

In [1]:
class Solution:
    def getSum(self, a: int, b: int) -> int:
        # 1. 設置遮罩 (Mask)：限制在 32 位元內 (0xFFFFFFFF = 32個1)
        # 這是為了模擬 C++/Java 的溢位行為，防止 Python 數字無限長大
        mask = 0xFFFFFFFF
        
        # 只要還有「進位」要處理，就繼續運算
        while b != 0:
            # a_new 代表「不進位加法」結果
            # 使用 XOR (^) 運算：相同為 0，不同為 1
            a_new = (a ^ b) & mask
            
            # carry 代表「進位」結果
            # 使用 AND (&) 找出需要進位的地方，並左移一位 (<< 1)
            carry = ((a & b) << 1) & mask
            
            # 更新 a 和 b，進入下一輪「能量分配」
            a = a_new
            b = carry
            
        # 2. 處理 Python 的負數表示法
        # 0x7FFFFFFF 是 32 位元能表達的最大正數
        # 如果 a 超過這個數，代表它的第 31 位是 1，在電腦裡它是個負數
        # 我們必須用 ~(a ^ mask) 把這個「大正數」轉回 Python 認得的「負數」
        return a if a <= 0x7FFFFFFF else ~(a ^ mask)

In [2]:
a = 7
b = 1
# Output: 3

Solution().getSum(a, b)

8

#### 一、 位元運算基礎 (Bitwise Basics)

| 符號 | 名稱 | 功能 | 在加法中的意義 |
| :--- | :--- | :--- | :--- |
| `^` | **XOR** | 相同為 0，不同為 1 | **不進位加法** (1+1=0, 1+0=1) |
| `&` | **AND** | 兩者皆為 1 才是 1 | **找進位位置** (只有 1+1 會產生進位) |
| `<< 1` | **左移** | 所有位元向左推一格 | **處理進位** (把進位加到更高的一位) |

---

#### 二、電腦底層邏輯

1. **32 位元整數結構**：
   - 一般電腦的 32 位元有號整數（Signed Integer）中，數值部分由 31 位組成。
   - **符號位元 (Sign Bit)**：最左邊的那一格（第 31 位）是關鍵。如果它是 `1`，代表的是 $-2^{31}$，整個數值會被解釋為負數。`0`則回正數。

2. **Python 的取反邏輯 (~x)**：
   - **公式**：`~x = -(x + 1)`。
   - **觀察**：這是因為正數是從 `0` 開始，但負數是從 `-1` 開始，存在一個位移。
   - **實例**：若 `x = 2` (二進位 `...010`)，取反後會對應到 `-3` (二進位 `...101`)。

3. **Python 無限位元的表示法**：
   - Python 的負數在邏輯上左邊有無窮多個 `1`。
   - **計算機實現**：Python 內部以物件儲存整數，會動態標記符號位。當進行位元運算時，會「模擬」出無限個 `1` 的行為，直到我們使用 `mask` (0xFFFFFFFF) 強行切斷。

---

#### 三、 核心邏輯驗證 (Verified Intuitions)

##### 1. 能量守恆定律
* **邏輯**：在每一輪迴圈中，`a + b` 的數值總和 $n$ 是固定的。我們只是不斷將「不進位的部分」與「進位的部分」重新分配，直到進位被完全處理。

##### 2. 進位推移與 B 的歸零
* **邏輯**：
    * **B 為什麼變大？** 因為 `b` 負責進位，每次 `<< 1` 都會將 1 往高位元推（數值加倍），所以看起來會突然變大。
    * **B 為什麼歸零？** 因為進位終究會找到 `a` 的空位（位元 0）停下來，或是衝出 32 位元的邊界而被遮罩切斷。
* **特殊路徑**：最糟情況下，會經歷 `A=0, B=n`（倒數第二步），最後一步則是 `A=n, B=0`。

---

#### 四、 Python 特性與負數處理

##### 1. 32 位元結構與 0x 標記
* **0x**：代表十六進位。
* **F**：代表4個1 (1111)。
* **0x7FFFFFFF**：32 位元中有號整數的最大正數（首位為 0，其餘 31 位為 1）。
* **0xFFFFFFFF (Mask)**：32 位元全為 1。用來「切斷」Python 無限長的位元，模擬硬體環境。

##### 2. 負數轉換邏輯
* **Python 負數公式**：`~x = -(x + 1)`。
* **轉換流程**：當 `a > 0x7FFFFFFF` 時，代表該數在 32 位元下是負數。
    1. `a ^ mask`：翻轉 32 位元內的所有 0 與 1。
    2. `~`：利用 Python 補數公式將翻轉後的結果轉回帶有 `-` 號的 Python 整數。